https://www.kaggle.com/pierremegret/gensim-word2vec-tutorial

# <b> Natural Language Processing and Word Embeddings using Spacy and Gensim

## <b> Word2Vec

Two words sharing similar contexts also share a similar meaning and consequently a similar vector representation from the model.  

Word2Vec can be used to find out the relations between words in a dataset, compute the similarity between them, or use 

the vector representation of those words as input for other applications such as text classification or clustering.

### <b> Imports

In [1]:
import re 
import pandas as pd  
from time import time 
from collections import defaultdict
import spacy 

from src import PreProcessing

### <b> Loading Data

In [2]:
df = pd.read_csv('data/cpfl.csv')

In [3]:
df['Clean Sentences'] = df['Conteúdo'].apply(lambda x: PreProcessing.clean_text(x))

In [4]:
df[['Título', 'Conteúdo', 'Clean Sentences']].head()

,Título,Conteúdo,Clean Sentences
0,Após frocar poste de energia elétrica abasteci...,Após frocar poste de energia elétrica abasteci...,apos frocar poste energia eletrica abastecimen...
1,Brasileiro pode pagar menos por energia elétrica,"Desde ontem, todos os consumidores brasileiros...",desde ontem todos consumidores brasileiros ja ...
2,CPFL abre 28 vagas para curso de eletricista e...,A CPFL Paulista abriu 28 vagas para o curso gr...,cpfl paulista abriu vagas curso gratuito forma...
3,"CPFL tem lucro líquido de R$ 904,126 milhões n...","A CPFL apurou lucro líquido de R$ 904,126 milh...",cpfl apurou lucro liquido milhoes primeiro tri...
4,Banco de preços de distribuição terá nova fase...,A Agência Nacional de Energia Elétrica vai abr...,agencia nacional energia eletrica vai abrir co...


## <b> Processing Data with Spacy

#### Portuguese Model

In [ ]:
nlp = spacy.load('pt_core_news_sm', disable=['ner', 'parser']) 

#### Cleaning

In [ ]:
def cleaning(sentence):
    text = [token.lemma_ for token in sentence if not token.is_stop]
    # filter small sentences
    if len(text) > 2:
        return ' '.join(text)

In [ ]:
title_cleaning = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in df['Título'])
text_cleaning = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in df['Conteúdo'])

In [ ]:
# NLP pipeline speeds up the time processing
title = [cleaning(doc) for doc in nlp.pipe(title_cleaning, batch_size=5000)]
text = [cleaning(doc) for doc in nlp.pipe(text_cleaning, batch_size=5000)]

In [ ]:
df_clean = pd.DataFrame({'clean_text': text, 'clean_title':title})
df_clean = df_clean.dropna().drop_duplicates()
df_clean.shape

In [ ]:
df_clean.head()

# <b> Ngrams

In [5]:
from gensim.models.phrases import Phrases, Phraser

In [6]:
sent = [row.split() for row in df['Clean Sentences']]

In [7]:
# removing small words
sent = [[x for x in line if len(x) > 2] for line in sent ] 

In [8]:
phrases = Phrases(sent, min_count=30, progress_per=100)

In [9]:
bigram = Phraser(phrases)

In [10]:
sentences = bigram[sent]

In [11]:
for sent in sentences:
    print(sent)
    break

['apos', 'frocar', 'poste', 'energia_eletrica', 'abastecimento_agua', 'restabelecido', 'zona', 'leste', 'apos', 'energia', 'paulista', 'cascata', 'troca', 'poste', 'eletrica', 'cpfl', 'ontem', 'bairro', 'daem', 'departamento', 'agua_esgoto', 'marilia', 'informou', 'abastecimento_agua', 'zona', 'leste', 'cidade', 'sera', 'restabelecido', 'hoje', 'manutencao', 'cpfl_paulista', 'aconteceu', 'diante', 'ocorrido', 'sistema', 'abastecimento', 'cascata', 'ficou', 'inativo', 'aproximadamente', 'seis', 'horas', 'atraves', 'redes_sociais', 'daem', 'pediu', 'economia', 'daem', 'informou', 'abastecimento_agua', 'sera', 'restabelecido', 'hoje', 'zona', 'leste', 'cidade', 'agua', 'moradores', 'regiao', 'leste', 'marilia', 'evitar', 'faltasse', 'ainda', 'produto', 'direcao', 'autarquia', 'destacou', 'residencias', 'reservatorios', 'agua', 'podem', 'ter', 'sentido', 'baixa', 'provocada', 'troca', 'poste', 'energia_eletrica', 'cpfl', 'divulgacao']


#### Most Frequent Words

In [12]:
word_freq = defaultdict(int)
for sent in sentences:
    for i in sent:
        word_freq[i] += 1
len(word_freq)

80670

In [13]:
print(sorted(word_freq, key=word_freq.get, reverse=True)[:10])

['nao', 'energia', 'brasil', 'empresa', 'tambem', 'empresas', 'ate', 'ainda', 'sao', 'sobre']


## <b> Training Gensim

In [14]:
import multiprocessing
from gensim.models import Word2Vec

In [15]:
cores = multiprocessing.cpu_count()
print(cores)

12


Parameters:

min_count = int - Ignores all words with total absolute frequency lower than this - (2, 100)

window = int - The maximum distance between the current and predicted word within a sentence. E.g. window words on the left and window words on the left of our target - (2, 10)

size = int - Dimensionality of the feature vectors. - (50, 300)

sample = float - The threshold for configuring which higher-frequency words are randomly downsampled. Highly influencial. - (0, 1e-5)

alpha = float - The initial learning rate - (0.01, 0.05)

min_alpha = float - Learning rate will linearly drop to min_alpha as training progresses. To set it: alpha - (min_alpha * epochs) ~ 0.00

negative = int - If > 0, negative sampling will be used, the int for negative specifies how many "noise words" should be drown. If set to 0, no negative 
sampling is used. - (5, 20)

workers = int - Use these many worker threads to train the model (=faster training with multicore machines)

In [16]:
w2v_model = Word2Vec(min_count=2,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

In [17]:
t = time()

w2v_model.build_vocab(sentences, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

Time to build vocab: 0.19 mins


In [18]:
t = time()

w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

Time to train the model: 2.19 mins


In [19]:
# memory efficient
w2v_model.init_sims(replace=True)

## <b> Exploring the model

#### Positives

In [20]:
w2v_model.wv.most_similar(positive=["energia"])

[('energia_eletrica', 0.6388442516326904),
 ('consumo', 0.4352198839187622),
 ('eletricidade', 0.40962857007980347),
 ('tambem', 0.3987608253955841),
 ('rge', 0.390339195728302),
 ('distribuidora', 0.3876175880432129),
 ('eletrica', 0.37769490480422974),
 ('eneigia', 0.3771359920501709),
 ('distribuicao', 0.3770790696144104),
 ('cpfl', 0.3681311309337616)]

#### Negatives

In [21]:
w2v_model.wv.most_similar(negative=["energia"])

[('tek', 0.3303612470626831),
 ('eve', 0.32560375332832336),
 ('widc', 0.3214746117591858),
 ('mont', 0.321318656206131),
 ('beve', 0.31915658712387085),
 ('haea', 0.31310516595840454),
 ('asm', 0.3127145767211914),
 ('bad', 0.31271427869796753),
 ('ampon', 0.3116723299026489),
 ('discovery', 0.311638742685318)]

#### Similarities

In [22]:
w2v_model.wv.similarity("luz", "energia")

0.29844242

In [23]:
w2v_model.wv.similarity("eletricidade", "energia")

0.4096286

In [24]:
w2v_model.wv.similarity("eletricidade", "luz")

0.2099957

In [25]:
w2v_model.wv.similarity("eletricidade", "eletrica")

0.2374003

## <b> t-SNE visualizations

In [44]:
from src import TSNE

In [45]:
words = list(w2v_model.wv.most_similar(positive=["luz"], topn=20))
words = [w[0] for w in words]

In [46]:
print(words)

['paulista_forca', 'restabeleca', 'reduzir_conta', 'caragua', 'clayton', 'feches', 'dmae', 'precos_consumidores', 'bebe_sexo', 'ultravioleta', 'relogio', 'bal', 'arquibancadas', 'cortada', 'moradoresde', 'persiste', 'arrancados', 'impactar_contas', 'incide', 'equipados']


In [ ]:
TSNE.tsnescatterplot(w2v_model, 'luz', words)

### <b> Save Embeddings Model

In [31]:
w2v_model.save("models/word2vec.model")

### <b> Save Word Vectors

In [32]:
word_vectors = w2v_model.wv

In [33]:
word_vectors.save("embeddings/word2vec.wordvectors")

### <b> Load Word Vectors

In [ ]:
from gensim.models import KeyedVectors
wv = KeyedVectors.load("embeddings/word2vec.wordvectors", mmap='r')

In [ ]:
vector = wv['luz']  # Get numpy vector of a word

In [49]:
df.to_csv('data/cpfl_clean.csv', index=False)

In [54]:
df['Clean Sentences'][1]

'desde ontem todos consumidores brasileiros ja podem aderir tarifa branca permite pagamento valores diferentes funcao hora dia semana consome energia eletrica aprovada agencia nacional energia eletrica aneel havia estabelecido cronograma anual adocao modalidade comecando janeiro novas ligacoes clientes unidades consumidoras media anual consumo mensal superior kwh mes janeiro patamar minimo reduzido kwh mes chegando agora todos consumidores baixa tensao tarifa branca medida adotada aneel promover sinal precos consumidores reduzir conta luz alem otimizar uso rede eletrica dias uteis nova modalidade tarifaria tres valores ponta intermediario ponta periodos sao estabelecidos aneel diferentes cada distribuidora sabados domingos feriados contam tarifa ponta horas dia regras valor tarifa horario ponta barato horario ponta estimulando mudanca habitos consumo perfil antes optar tarifa branca consumidor deve conhecer bem perfil consumo quanto deslocar consumo periodo ponta maior sera reducao con

In [55]:
df['Conteúdo'][1]

'Desde ontem, todos os consumidores brasileiros já podem aderir à tarifa branca, que permite o pagamento de valores diferentes em função da hora e do dia da semana em que se consome a energia elétrica. Aprovada em 2016, a Agência Nacional de Energia Elétrica (Aneel) havia estabelecido um cronograma anual para adoção da modalidade, começando em janeiro de 2018 pelas novas ligações de clientes ou unidades consumidoras com média anual de consumo mensal superior a 500 KWh/mês. Em janeiro de 2019, o patamar mínimo foi reduzido para 250 KWh/mês, chegando a agora a todos os consumidores de baixa tensão.\nA tarifa branca é uma medida adotada pela Aneel para promover o sinal de preços aos consumidores e reduzir a conta de luz, além de otimizar o uso da rede elétrica. Nos dias úteis, a nova modalidade tarifária tem três valores: ponta, intermediário e fora de ponta. Esses períodos são estabelecidos pela Aneel e diferentes para cada distribuidora. Sábados, domingos e feriados contam com a tarifa 

In [52]:
df[['Título', 'Conteúdo', 'Clean Sentences']].head()

,Título,Conteúdo,Clean Sentences
0,Após frocar poste de energia elétrica abasteci...,Após frocar poste de energia elétrica abasteci...,apos frocar poste energia eletrica abastecimen...
1,Brasileiro pode pagar menos por energia elétrica,"Desde ontem, todos os consumidores brasileiros...",desde ontem todos consumidores brasileiros ja ...
2,CPFL abre 28 vagas para curso de eletricista e...,A CPFL Paulista abriu 28 vagas para o curso gr...,cpfl paulista abriu vagas curso gratuito forma...
3,"CPFL tem lucro líquido de R$ 904,126 milhões n...","A CPFL apurou lucro líquido de R$ 904,126 milh...",cpfl apurou lucro liquido milhoes primeiro tri...
4,Banco de preços de distribuição terá nova fase...,A Agência Nacional de Energia Elétrica vai abr...,agencia nacional energia eletrica vai abrir co...
